In [2]:
import os
import sys

import findspark
findspark.init()

In [3]:
import pandas as pd
from pyspark.mllib.clustering import KMeans, KMeansModel
from numpy import array

In [4]:
ac_data = pd.read_csv("C:/Data-Attraper le flamand rose/ad-clicks.csv")

In [5]:
##nous voulons une vision d'ensemble des données 
ac_data.head()

,timestamp,txId,userSessionId,teamId,userId,adId,adCategory
0,2016-05-26 15:13:22,5974,5809,27,611,2,electronics
1,2016-05-26 15:17:24,5976,5705,18,1874,21,movies
2,2016-05-26 15:22:52,5978,5791,53,2139,25,computers
3,2016-05-26 15:22:57,5973,5756,63,212,10,fashion
4,2016-05-26 15:22:58,5980,5920,9,1027,20,clothing


In [6]:
##ajout d'un compteur 
ac_data['adCompteur'] = 1

In [7]:
ac_data.head()

,timestamp,txId,userSessionId,teamId,userId,adId,adCategory,adCompteur
0,2016-05-26 15:13:22,5974,5809,27,611,2,electronics,1
1,2016-05-26 15:17:24,5976,5705,18,1874,21,movies,1
2,2016-05-26 15:22:52,5978,5791,53,2139,25,computers,1
3,2016-05-26 15:22:57,5973,5756,63,212,10,fashion,1
4,2016-05-26 15:22:58,5980,5920,9,1027,20,clothing,1


In [8]:
gc_data = pd.read_csv("C:/Data-Attraper le flamand rose/game-clicks.csv")

In [9]:
gc_data['gcCompteur'] = 1
gc_data.head()

,timestamp,clickId,userId,userSessionId,isHit,teamId,teamLevel,gcCompteur
0,2016-05-26 15:06:55,105,1038,5916,0,25,1,1
1,2016-05-26 15:07:09,154,1099,5898,0,44,1,1
2,2016-05-26 15:07:14,229,899,5757,0,71,1,1
3,2016-05-26 15:07:14,322,2197,5854,0,99,1,1
4,2016-05-26 15:07:20,22,1362,5739,0,13,1,1


In [10]:
bc_data = pd.read_csv("C:/Data-Attraper le flamand rose/buy-clicks.csv")
bc_data.head()

,timestamp,txId,userSessionId,team,userId,buyId,price
0,2016-05-26 15:36:54,6004,5820,9,1300,2,3.0
1,2016-05-26 15:36:54,6005,5775,35,868,4,10.0
2,2016-05-26 15:36:54,6006,5679,97,819,5,20.0
3,2016-05-26 16:36:54,6067,5665,18,121,2,3.0
4,2016-05-26 17:06:54,6093,5709,11,2222,5,20.0


In [11]:
u_purchases = bc_data[['userId','price']] #on selectionne uniquement l'userid et le prix
u_purchases.head()

,userId,price
0,1300,3.0
1,868,10.0
2,819,20.0
3,121,3.0
4,2222,20.0


In [12]:
u_purchases = bc_data[['userId','price']] #on selectionne uniquement l'userid et le prix
u_purchases.head()

,userId,price
0,1300,3.0
1,868,10.0
2,819,20.0
3,121,3.0
4,2222,20.0


In [13]:
u_aclicks = ac_data[['userId','adCompteur']]
u_aclicks.head()

,userId,adCompteur
0,611,1
1,1874,1
2,2139,1
3,212,1
4,1027,1


In [14]:
u_gameClicks = gc_data[['userId','gcCompteur']]
u_gameClicks.head()

,userId,gcCompteur
0,1038,1
1,1099,1
2,899,1
3,2197,1
4,1362,1


In [16]:
user_ads = u_aclicks.groupby('userId').sum()
user_ads = user_ads.reset_index()
user_ads.columns = ['userId', 'acdataTotal']
user_ads.head()

,userId,acdataTotal
0,1,44
1,8,10
2,9,37
3,10,19
4,12,46


In [18]:
user_gc_data = u_gameClicks.groupby('userId').sum()
user_gc_data = user_gc_data.reset_index()
user_gc_data.columns = ['userId', 'gcdataTotal']
user_gc_data.head()

,userId,gcdataTotal
0,0,1355
1,1,716
2,2,231
3,6,151
4,8,380


In [19]:
user_benefice = u_purchases.groupby('userId').sum()
user_benefice = user_benefice.reset_index()
user_benefice.columns = ['userId', 'benefice']
user_benefice.head()

,userId,benefice
0,1,21.0
1,8,53.0
2,9,80.0
3,10,11.0
4,12,215.0


In [20]:
data_combinaison = user_ads.merge(user_gc_data, on='userId') 
data_combinaison = data_combinaison.merge(user_benefice, on='userId') 
data_combinaison.head()

,userId,acdataTotal,gcdataTotal,benefice
0,1,44,716,21.0
1,8,10,380,53.0
2,9,37,508,80.0
3,10,19,3107,11.0
4,12,46,704,215.0


In [21]:
data_clustering = data_combinaison[['acdataTotal', 'gcdataTotal','benefice']]
data_clustering.head()

,acdataTotal,gcdataTotal,benefice
0,44,716,21.0
1,10,380,53.0
2,37,508,80.0
3,19,3107,11.0
4,46,704,215.0


In [22]:
data_clustering.shape

(543, 3)

In [30]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("sample_app")
sc = SparkContext(conf=conf)

In [31]:
#L'application de notre algorithme KMEANS necessite la conversion de notre table en un format comprehensible par le KNeans.train
sqlContext = SQLContext(sc)
training_df= sqlContext.createDataFrame(data_clustering)
training_data = training_df.rdd.map(lambda line: array([line[0], line[1], line[2]]))

In [34]:
#on cree notre model avec un nombre  de cluster = 3
model = KMeans.train(training_data , 3, maxIterations=10, runs=10, initializationMode="random")

In [35]:
print(model.centers)

[array([  32.35555556, 2310.64444444,   39.42222222]), array([ 24.98746082, 357.95924765,  35.06583072]), array([ 36.44134078, 926.11731844,  46.96648045])]
